In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [6]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [11]:
# unique launch sites
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()

# base map (centered on average coords)
center = [launch_sites['Lat'].mean(), launch_sites['Long'].mean()]
site_map = folium.Map(location=center, zoom_start=4)

# add markers
for _, row in launch_sites.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site']
    ).add_to(site_map)

site_map

In [12]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [13]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [14]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


In [15]:
# Base map
center = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]
site_map = folium.Map(location=center, zoom_start=4)

# Add markers for each launch
for _, row in spacex_df.iterrows():
    
    if row['class'] == 1:
        color = 'green'
    else:
        color = 'red'
    
    folium.CircleMarker(
        location=[row['Lat'], row['Long']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        popup=row['Launch Site']
    ).add_to(site_map)

site_map

In [16]:
# Create marker_color column based on class value
spacex_df['marker_color'] = spacex_df['class'].apply(
    lambda x: 'green' if x == 1 else 'red'
)

spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [18]:
marker_cluster = MarkerCluster()


In [19]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [20]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [21]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [23]:


# Get one launch site (example: first unique site)
launch_site = spacex_df[['Launch Site','Lat','Long']].drop_duplicates().iloc[0]

launch_site_lat = launch_site['Lat']
launch_site_lon = launch_site['Long']


# Closest coastline coordinates (example provided)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance
distance_coastline = calculate_distance(
    launch_site_lat,
    launch_site_lon,
    coastline_lat,
    coastline_lon
)

print("Distance to closest coastline (km):", distance_coastline)

Distance to closest coastline (km): 0.5797581813109574


In [24]:
# Coordinates list (launch site → coastline)
coordinates = [
    [launch_site_lat, launch_site_lon],
    [coastline_lat, coastline_lon]
]

# Create PolyLine
lines = folium.PolyLine(
    locations=coordinates,
    weight=2,
    color='blue'
)

# Add to map
site_map.add_child(lines)

site_map

In [25]:
# Example proximity point (replace with your closest city / railway / highway coords)
proximity_name = "Closest City"
proximity_lat = 28.56
proximity_lon = -80.58

# Distance (km)
distance_proximity = calculate_distance(
    launch_site_lat, launch_site_lon,
    proximity_lat, proximity_lon
)

# Marker for proximity point
folium.Marker(
    location=[proximity_lat, proximity_lon],
    popup=f"{proximity_name} ({distance_proximity:.2f} km)",
    icon=folium.Icon(color="purple")
).add_to(site_map)

# Line from launch site to proximity point
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [proximity_lat, proximity_lon]],
    weight=2,
    color="purple"
).add_to(site_map)

site_map

In [26]:
import json, base64
from IPython.display import HTML
import IPython

nb = IPython.get_ipython().parent.get('metadata', None)


<class 'AttributeError'>: 'NoneType' object has no attribute 'get'